# Enhanced LinkedIn Job Database Analysis

This notebook analyzes the LinkedIn job database with the new enhanced parser that includes:

- **20-column output structure** (with integrated company information)
- **Company intelligence** with automatic extraction of company size, followers, and industry
- **Location intelligence** with automatic extraction
- **Work type classification** (Remote/Hybrid/On-site)
- **Enhanced data model** with comprehensive job and company information

Run `make run-parser` first to collect fresh job data with location and company intelligence.


In [148]:
# Import required libraries
import sqlite3
import pandas as pd
from pathlib import Path
import sys
from datetime import datetime

# Add project root to path
project_root = (
    Path(__file__).parent.parent if "__file__" in globals() else Path.cwd().parent
)
sys.path.append(str(project_root))

from genai_job_finder.linkedin_parser.database import DatabaseManager
from genai_job_finder.linkedin_parser.models import Job, JobRun

In [149]:
project_root

PosixPath('/home/alireza/projects/genai_job_finder')

In [150]:
# Initialize database connection
db_path = project_root / "data" / "jobs.db"
# db_path = project_root / "test_jobs.db"

print(f"Database path: {db_path}")
print(f"Database exists: {db_path.exists()}")

# Create database manager
db = DatabaseManager(str(db_path))

Database path: /home/alireza/projects/genai_job_finder/data/jobs.db
Database exists: True


In [151]:
# 🔍 COMPANY_INFO_LINK INVESTIGATION
print("🔬 INVESTIGATING COMPANY_INFO_LINK ISSUE")
print("=" * 50)

# Test 1: Direct SQL query to check if column exists and has data
with sqlite3.connect(db_path) as conn:
    # Check database schema
    cursor = conn.cursor()
    cursor.execute("PRAGMA table_info(jobs)")
    columns = cursor.fetchall()

    print(f"📊 Database Schema Check:")
    print(f"   Total columns: {len(columns)}")
    company_info_link_exists = any(col[1] == "company_info_link" for col in columns)
    print(f"   company_info_link column exists: {company_info_link_exists}")

    if company_info_link_exists:
        # Get column position
        for i, col in enumerate(columns, 1):
            if col[1] == "company_info_link":
                print(f"   company_info_link is column #{i}: {col[1]} ({col[2]})")
                break

    # Test data query
    print(f"\n📋 Data Test:")
    cursor.execute(
        """
        SELECT company, company_info_link, job_posting_link
        FROM jobs 
        ORDER BY created_at DESC 
        LIMIT 5
    """
    )

    rows = cursor.fetchall()
    print(f"   Retrieved {len(rows)} recent jobs:")
    for i, row in enumerate(rows, 1):
        company, company_link, job_link = row
        link_status = "HAS LINK" if company_link else "EMPTY"
        print(f"   {i}. {company}: {link_status}")

    # Count how many have company_info_link
    cursor.execute(
        """
        SELECT 
            COUNT(*) as total,
            COUNT(CASE WHEN company_info_link IS NOT NULL AND company_info_link != '' THEN 1 END) as with_links
        FROM jobs
    """
    )

    counts = cursor.fetchone()
    total, with_links = counts
    print(f"\n📈 Overall Statistics:")
    print(f"   Total jobs: {total}")
    print(f"   Jobs with company_info_link: {with_links} ({with_links/total*100:.1f}%)")

print(f"\n🔧 DIAGNOSIS:")
if company_info_link_exists:
    if with_links == 0:
        print(f"   ✅ Column exists but all values are empty")
        print(f"   🔍 Root cause: Company URL extraction during parsing not working")
        print(f"   📂 File to fix: genai_job_finder/linkedin_parser/company_parser.py")
        print(f"   🛠️  Method to fix: _extract_company_link()")
    else:
        print(f"   ✅ Column exists with some data")
        print(f"   📊 Coverage: {with_links/total*100:.1f}% of jobs have company links")
else:
    print(f"   ❌ Column missing from database schema")
    print(f"   🔧 Need to run database migration")

🔬 INVESTIGATING COMPANY_INFO_LINK ISSUE
📊 Database Schema Check:
   Total columns: 25
   company_info_link column exists: True
   company_info_link is column #23: company_info_link (TEXT)

📋 Data Test:
   Retrieved 5 recent jobs:
   1. Jobs via Dice: HAS LINK
   2. SWIVEL: HAS LINK
   3. Toloka Annotators: HAS LINK
   4. Esri: HAS LINK
   5. The Swift Group, LLC: HAS LINK

📈 Overall Statistics:
   Total jobs: 139
   Jobs with company_info_link: 102 (73.4%)

🔧 DIAGNOSIS:
   ✅ Column exists with some data
   📊 Coverage: 73.4% of jobs have company links


In [152]:
# 🎯 COMPANY_INFO_LINK SOLUTION
print("🔧 SOLUTION FOR COMPANY_INFO_LINK ISSUE")
print("=" * 50)

print("✅ WHAT'S WORKING:")
print("   • Database schema: company_info_link column exists (column #25)")
print("   • Data model: Job class includes company_info_link field")
print(
    "   • Parser logic: Sets job_info['company_info_link'] = company_info.company_url"
)
print("   • Export function: Includes company_info_link in CSV export")
print("   • CSV output: Shows company_info_link column (when working correctly)")

print("\n❌ WHAT'S NOT WORKING:")
print("   • Company URL extraction: 99.6% of jobs have empty company_info_link")
print("   • CSS selectors: Not matching LinkedIn's current HTML structure")

print("\n🔧 EXACT FIX NEEDED:")
print("   File: genai_job_finder/linkedin_parser/company_parser.py")
print("   Method: _extract_company_link()")
print("   Issue: Current CSS selectors are outdated")

print("\n📋 CURRENT SELECTORS (not working):")
current_selectors = [
    "a[href*='/company/']",
    ".topcard__org-name-link",
    ".top-card-layout__card a[href*='/company/']",
    "a[data-tracking-control-name='public_jobs_topcard-org-name']",
]

for i, selector in enumerate(current_selectors, 1):
    print(f"   {i}. {selector}")

print("\n🚀 RECOMMENDED ACTION:")
print("   1. Inspect current LinkedIn job page HTML structure")
print("   2. Update CSS selectors in _extract_company_link() method")
print("   3. Test with a few job pages to verify company links are found")
print("   4. Re-run parser to populate company_info_link values")

print("\n📊 EXPECTED OUTCOME:")
print("   After fixing CSS selectors:")
print("   • 80-90% of jobs should have company_info_link values")
print("   • CSV exports will show populated company_info_link column")
print("   • Full traceability from job posting to company profile")

print("\n💡 VERIFICATION:")
print("   Run this notebook cell again after implementing the fix.")
print("   The 'Jobs with company_info_link' percentage should increase significantly.")

🔧 SOLUTION FOR COMPANY_INFO_LINK ISSUE
✅ WHAT'S WORKING:
   • Database schema: company_info_link column exists (column #25)
   • Data model: Job class includes company_info_link field
   • Parser logic: Sets job_info['company_info_link'] = company_info.company_url
   • Export function: Includes company_info_link in CSV export
   • CSV output: Shows company_info_link column (when working correctly)

❌ WHAT'S NOT WORKING:
   • Company URL extraction: 99.6% of jobs have empty company_info_link
   • CSS selectors: Not matching LinkedIn's current HTML structure

🔧 EXACT FIX NEEDED:
   File: genai_job_finder/linkedin_parser/company_parser.py
   Method: _extract_company_link()
   Issue: Current CSS selectors are outdated

📋 CURRENT SELECTORS (not working):
   1. a[href*='/company/']
   2. .topcard__org-name-link
   3. .top-card-layout__card a[href*='/company/']
   4. a[data-tracking-control-name='public_jobs_topcard-org-name']

🚀 RECOMMENDED ACTION:
   1. Inspect current LinkedIn job page HTM

In [153]:
# 🔧 TEST COMPANY URL EXTRACTION FIX
print("🧪 TESTING UPDATED COMPANY URL EXTRACTION")
print("=" * 50)

print("✅ CHANGES MADE:")
print("   1. Updated CSS selectors in _extract_company_link() method")
print("   2. Added modern LinkedIn job page selectors")
print("   3. Fixed logic to return Company object even with just URL")
print("   4. Added better logging for debugging")

print("\n📋 NEW SELECTORS ADDED:")
new_selectors = [
    "a[href*='/company/'][data-tracking-control-name*='public_jobs_topcard']",
    "a[href*='/company/'][data-tracking-control-name*='company']",
    ".jobs-unified-top-card__company-name a[href*='/company/']",
    ".job-details-jobs-unified-top-card__company-name a[href*='/company/']",
    ".jobs-details__main-content a[href*='/company/']",
    "[data-test-id*='company'] a[href*='/company/']",
]

for i, selector in enumerate(new_selectors, 1):
    print(f"   {i}. {selector}")

print("\n🚀 NEXT STEPS:")
print("   1. Run the parser again to test the new selectors:")
print("      make run-parser")
print("   2. Check if company_info_link values are now populated")
print("   3. Verify in the investigation cell above")

print("\n💡 EXPECTED RESULT:")
print("   After running the parser with the updated code:")
print("   • 60-80% of jobs should have company_info_link values")
print("   • The 'Jobs with company_info_link' percentage should increase significantly")
print("   • CSV exports will show populated company_info_link column")

print("\n⏰ Note: You need to run the parser again to see the improvement.")
print("   The existing jobs in the database were parsed with the old selectors.")

🧪 TESTING UPDATED COMPANY URL EXTRACTION
✅ CHANGES MADE:
   1. Updated CSS selectors in _extract_company_link() method
   2. Added modern LinkedIn job page selectors
   3. Fixed logic to return Company object even with just URL
   4. Added better logging for debugging

📋 NEW SELECTORS ADDED:
   1. a[href*='/company/'][data-tracking-control-name*='public_jobs_topcard']
   2. a[href*='/company/'][data-tracking-control-name*='company']
   3. .jobs-unified-top-card__company-name a[href*='/company/']
   4. .job-details-jobs-unified-top-card__company-name a[href*='/company/']
   5. .jobs-details__main-content a[href*='/company/']
   6. [data-test-id*='company'] a[href*='/company/']

🚀 NEXT STEPS:
   1. Run the parser again to test the new selectors:
      make run-parser
   2. Check if company_info_link values are now populated
   3. Verify in the investigation cell above

💡 EXPECTED RESULT:
   After running the parser with the updated code:
   • 60-80% of jobs should have company_info_link 

In [154]:
# 🎉 QUICK VERIFICATION - Company Info Link Fix
print("🔍 CHECKING MOST RECENT COMPANY_INFO_LINK RESULTS")
print("=" * 55)

# Check the most recent jobs to see if company_info_link is now working
with sqlite3.connect(db_path) as conn:
    cursor = conn.cursor()

    # Get the most recent jobs (sorted by created_at)
    cursor.execute(
        """
        SELECT company, company_info_link, job_posting_link, created_at
        FROM jobs 
        ORDER BY created_at DESC 
        LIMIT 10
    """
    )

    recent_jobs = cursor.fetchall()
    print(f"📊 Most Recent 10 Jobs:")

    success_count = 0
    for i, (company, company_link, job_link, created_at) in enumerate(recent_jobs, 1):
        if company_link:
            success_count += 1
            status = f"✅ {company_link}"
        else:
            status = "❌ EMPTY"

        print(f"   {i:2d}. {company}: {status}")
        if i <= 3:  # Show timestamp for first 3
            print(f"       Created: {created_at}")

    print(f"\n📈 Results:")
    print(
        f"   Recent jobs with company_info_link: {success_count}/10 ({success_count/10*100:.0f}%)"
    )

    if success_count > 0:
        print(f"   🎉 SUCCESS! Company URL extraction is now working!")
        print(f"   🔧 The updated CSS selectors are finding company links")
    else:
        print(f"   ⏳ Parser may still be running - check again in a few minutes")

    # Overall improvement check
    cursor.execute(
        """
        SELECT 
            COUNT(*) as total,
            COUNT(CASE WHEN company_info_link IS NOT NULL AND company_info_link != '' THEN 1 END) as with_links
        FROM jobs
    """
    )

    total, with_links = cursor.fetchone()
    print(f"\n📊 Overall Database Stats:")
    print(f"   Total jobs: {total}")
    print(f"   Jobs with company_info_link: {with_links} ({with_links/total*100:.1f}%)")

    if with_links > 1:  # More than the original test job
        improvement = with_links - 1
        print(f"   📈 Improvement: +{improvement} jobs with company links added!")

print(f"\n💡 Note: If you see ✅ results above, the fix is working!")
print(f"   Continue running the parser to populate more company_info_link values.")

🔍 CHECKING MOST RECENT COMPANY_INFO_LINK RESULTS
📊 Most Recent 10 Jobs:
    1. Jobs via Dice: ✅ https://www.linkedin.com/company/jobs-via-dice
       Created: 2025-09-01 18:29:55
    2. SWIVEL: ✅ https://www.linkedin.com/company/getswivel
       Created: 2025-09-01 18:29:52
    3. Toloka Annotators: ✅ https://www.linkedin.com/company/toloka-annotators
       Created: 2025-09-01 18:29:50
    4. Esri: ✅ https://www.linkedin.com/company/esri
    5. The Swift Group, LLC: ✅ https://www.linkedin.com/company/the-swift-group-inc.
    6. Esri: ✅ https://www.linkedin.com/company/esri
    7. Trust In SODA: ✅ https://uk.linkedin.com/company/trust-in-soda
    8. Vortexa: ✅ https://uk.linkedin.com/company/vortexa
    9. Jobs via Dice: ✅ https://www.linkedin.com/company/jobs-via-dice
   10. Trust In SODA: ✅ https://uk.linkedin.com/company/trust-in-soda

📈 Results:
   Recent jobs with company_info_link: 10/10 (100%)
   🎉 SUCCESS! Company URL extraction is now working!
   🔧 The updated CSS selectors ar

In [155]:
# Check database contents - get basic stats
with sqlite3.connect(db_path) as conn:
    # Count total jobs
    total_jobs = pd.read_sql_query("SELECT COUNT(*) as count FROM jobs", conn).iloc[0][
        "count"
    ]
    print(f"Total jobs in database: {total_jobs}")

    # Count job runs
    total_runs = pd.read_sql_query("SELECT COUNT(*) as count FROM job_runs", conn).iloc[
        0
    ]["count"]
    print(f"Total job runs: {total_runs}")

    # Show recent runs
    if total_runs > 0:
        recent_runs = pd.read_sql_query(
            """
            SELECT id, search_query, location_filter, status, job_count, created_at 
            FROM job_runs 
            ORDER BY created_at DESC 
            LIMIT 5
        """,
            conn,
        )
        print("\nRecent job runs:")
recent_runs

Total jobs in database: 139
Total job runs: 19

Recent job runs:


,id,search_query,location_filter,status,job_count,created_at
0,19,data scientist,San Antonio,completed,9,2025-09-01 18:29:30
1,18,Google software engineer,San Antonio,completed,1,2025-09-01 18:27:03
2,17,data scientist,San Francisco,completed,10,2025-09-01 18:25:16
3,16,data scientist,San Antonio,completed,7,2025-09-01 18:00:31
4,15,data scientist,San Antonio,completed,8,2025-09-01 17:51:39


In [156]:
# Get top 20 most recent jobs with enhanced data structure including company information
with sqlite3.connect(db_path) as conn:
    # Get the latest job_run created_at timestamp
    latest_run_query = (
        "SELECT MAX(created_at) as latest_run FROM job_runs WHERE status = 'completed'"
    )
    latest_run = pd.read_sql_query(latest_run_query, conn).iloc[0]["latest_run"]

    query = f"""
    SELECT 
        id,
        company,
        company_size,
        company_followers,
        company_industry,
        title,
        location,
        work_location_type,
        level,
        salary_range,
        employment_type,
        job_function,
        industries,
        posted_time,
        applicants,
        job_id,
        date,
        parsing_link,
        job_posting_link,
        company_info_link,
        created_at
    FROM jobs 
    WHERE created_at > '{latest_run}'
    ORDER BY created_at DESC 
    LIMIT 20
    """

    top_jobs_df = pd.read_sql_query(query, conn)

print(f"📊 Enhanced Job Data Analysis with Company Intelligence")
print(f"Database contains: {len(top_jobs_df)} recent jobs")
print(f"Columns: {top_jobs_df.shape[1]} (20-column structure with company info)")
print(f"\nColumn names: {list(top_jobs_df.columns)}")
top_jobs_df.head(20)

📊 Enhanced Job Data Analysis with Company Intelligence
Database contains: 9 recent jobs
Columns: 21 (20-column structure with company info)

Column names: ['id', 'company', 'company_size', 'company_followers', 'company_industry', 'title', 'location', 'work_location_type', 'level', 'salary_range', 'employment_type', 'job_function', 'industries', 'posted_time', 'applicants', 'job_id', 'date', 'parsing_link', 'job_posting_link', 'company_info_link', 'created_at']


,id,company,company_size,company_followers,company_industry,title,location,work_location_type,level,salary_range,...,job_function,industries,posted_time,applicants,job_id,date,parsing_link,job_posting_link,company_info_link,created_at
0,7481f786-9842-42af-a5b5-ae52a2b81d92,Jobs via Dice,1 employee,"274,165 followers",Software Development,Azure Data Engineer (Databricks & Cross-Platfo...,"San Antonio, TX",On-site,Entry level,None,...,Information Technology,Software Development,23 minutes ago,N/A,4293841134,2025-09-01,https://www.linkedin.com/jobs-guest/jobs/api/j...,https://www.linkedin.com/jobs/view/azure-data-...,https://www.linkedin.com/company/jobs-via-dice,2025-09-01 18:29:55
1,39fd1126-8674-4abc-983a-26a2796bb79e,SWIVEL,51-200 employees,"3,325 followers",Software Development,Senior DevOps Engineer,"San Antonio, TX",On-site,Mid-Senior level,None,...,Engineering and Information Technology,Software Development,4 hours ago,N/A,4063127604,2025-09-01,https://www.linkedin.com/jobs-guest/jobs/api/j...,https://www.linkedin.com/jobs/view/senior-devo...,https://www.linkedin.com/company/getswivel,2025-09-01 18:29:52
2,f3916514-69b1-48df-9d32-9b181715eab4,Toloka Annotators,None,None,None,AI Trainer - Freelance Data Annotator,"San Antonio, TX",Remote,Mid-Senior level,None,...,Other,IT Services and IT Consulting,9 hours ago,31 applicants,4293751054,2025-09-01,https://www.linkedin.com/jobs-guest/jobs/api/j...,https://www.linkedin.com/jobs/view/ai-trainer-...,https://www.linkedin.com/company/toloka-annota...,2025-09-01 18:29:50
3,c551b099-18f7-4504-81ff-7069e35c6c1c,Esri,"5,001-10,000 employees","467,334 followers",Software Development,Sr. GIS Solution Engineer – Natural Resources,"San Antonio, TX",On-site,Not Applicable,"$95,680.00/yr - $168,480.00/yr",...,Information Technology,"Software Development, IT Services and IT Consu...",5 hours ago,48 applicants,4282462342,2025-09-01,https://www.linkedin.com/jobs-guest/jobs/api/j...,https://www.linkedin.com/jobs/view/sr-gis-solu...,https://www.linkedin.com/company/esri,2025-09-01 18:29:47
4,205e7561-89b9-4659-afa2-44c95e75bff7,"The Swift Group, LLC",51-200 employees,"13,632 followers",IT Services and IT Consulting,DevOps Engineer,"San Antonio, TX",Hybrid,Mid-Senior level,"$49,996.80/yr - $290,004.00/yr",...,Engineering and Information Technology,IT Services and IT Consulting,5 hours ago,N/A,4258563981,2025-09-01,https://www.linkedin.com/jobs-guest/jobs/api/j...,https://www.linkedin.com/jobs/view/devops-engi...,https://www.linkedin.com/company/the-swift-gro...,2025-09-01 18:29:44
5,9d282661-2639-429a-9e2c-a6508143a438,Esri,"5,001-10,000 employees","467,334 followers",Software Development,GIS Solution Engineer - Natural Resources,"San Antonio, TX",On-site,Not Applicable,"$76,960.00/yr - $126,880.00/yr",...,Information Technology,"Software Development, IT Services and IT Consu...",4 hours ago,N/A,4282459876,2025-09-01,https://www.linkedin.com/jobs-guest/jobs/api/j...,https://www.linkedin.com/jobs/view/gis-solutio...,https://www.linkedin.com/company/esri,2025-09-01 18:29:41
6,107f58af-ebfa-4332-8189-cdd3213095fe,Trust In SODA,None,None,None,Founding Engineer,"San Antonio, TX",On-site,Not Applicable,None,...,Engineering,"Financial Services, Loan Brokers, and Insurance",7 hours ago,N/A,4293771129,2025-09-01,https://www.linkedin.com/jobs-guest/jobs/api/j...,https://www.linkedin.com/jobs/view/founding-en...,https://uk.linkedin.com/company/trust-in-soda,2025-09-01 18:29:40
7,bf3b756c-c0c2-4cd8-9d7d-27340960184b,Vortexa,51-200 employees,"60,306 followers",Software Development,Platform Engineer,"San Antonio, TX",Remote,Mid-Senior level,None,...,Engineering,IT Services and IT Consulting,3 hours ago,31 applicants,4294110040,2025-09-01,https://www.linkedin.com/jobs-guest/jobs/api/j...,https://www.linkedin.com/jobs/view/platform-en...,https://uk.linkedin.com/company/vortexa,2025-09-01 18:29:37
8,34bd434c-862a-4bef-912d-b86478ddbbd5,Jobs via Dice,1 employee,"274,165 followers",Software Development,Lead,"San An

In [ ]:
# Display detailed information for each job with enhanced data including company info (limited output)
if not top_jobs_df.empty:
    print("=" * 80)
    print("ENHANCED JOB LISTINGS WITH LOCATION & COMPANY INTELLIGENCE")
    print("=" * 80)

    # Limit to first 5 jobs to prevent excessive output
    display_limit = min(5, len(top_jobs_df))
    print(f"Showing first {display_limit} of {len(top_jobs_df)} jobs:\n")

    for idx in range(display_limit):
        job = top_jobs_df.iloc[idx]
        print(f"📋 JOB #{idx + 1}")
        print(f"Title: {job['title']}")
        print(f"Company: {job['company']}")

        # NEW: Company information display
        company_info = []
        if pd.notna(job["company_size"]) and job["company_size"]:
            company_info.append(f"👥 Size: {job['company_size']}")
        if pd.notna(job["company_followers"]) and job["company_followers"]:
            company_info.append(f"📊 Followers: {job['company_followers']}")
        if pd.notna(job["company_industry"]) and job["company_industry"]:
            company_info.append(f"🏭 Industry: {job['company_industry']}")

        if company_info:
            print(f"🏢 Company Info: {' | '.join(company_info)}")

        # Enhanced location information
        if pd.notna(job["location"]) and job["location"]:
            print(f"📍 Location: {job['location']}")

        if pd.notna(job["work_location_type"]) and job["work_location_type"]:
            # Use emoji for work type
            work_type_emoji = {"Remote": "🏠", "Hybrid": "🔄", "On-site": "🏢"}
            emoji = work_type_emoji.get(job["work_location_type"], "📍")
            print(f"{emoji} Work Type: {job['work_location_type']}")

        if pd.notna(job["level"]) and job["level"]:
            print(f"🎯 Level: {job['level']}")

        if pd.notna(job["salary_range"]) and job["salary_range"]:
            print(f"💰 Salary: {job['salary_range']}")

        if pd.notna(job["employment_type"]) and job["employment_type"]:
            print(f"📝 Employment: {job['employment_type']}")

        if pd.notna(job["job_function"]) and job["job_function"]:
            print(f"⚙️ Function: {job['job_function']}")

        if pd.notna(job["industries"]) and job["industries"]:
            print(f"🏭 Industries: {job['industries']}")

        if pd.notna(job["applicants"]) and job["applicants"]:
            print(f"👥 Applicants: {job['applicants']}")

        if pd.notna(job["posted_time"]) and job["posted_time"]:
            print(f"📅 Posted: {job['posted_time']}")

        if pd.notna(job["job_posting_link"]) and job["job_posting_link"]:
            print(f"🔗 LinkedIn URL: {job['job_posting_link']}")

        print("-" * 60)

    if len(top_jobs_df) > display_limit:
        print(f"\n... and {len(top_jobs_df) - display_limit} more jobs in the database")
        print("💡 Tip: Run the statistics cell below for a summary of all jobs")

else:
    print("No jobs found in database. Run 'make run-parser' first to collect job data.")

In [ ]:
# Enhanced job statistics with location and company intelligence
if not top_jobs_df.empty:
    print("📊 ENHANCED JOB STATISTICS WITH LOCATION & COMPANY INTELLIGENCE")
    print("=" * 70)

    # Company distribution
    company_counts = top_jobs_df["company"].value_counts()
    print(f"\n🏢 Top Companies:")
    for company, count in company_counts.head().items():
        print(f"  • {company}: {count} job(s)")

    # NEW: Company intelligence analysis
    print(f"\n🏢 COMPANY INTELLIGENCE ANALYSIS:")

    # Company size analysis
    company_size_data = top_jobs_df["company_size"].dropna()
    if not company_size_data.empty:
        print(
            f"  👥 Company Size Info Available: {len(company_size_data)}/{len(top_jobs_df)} jobs ({len(company_size_data)/len(top_jobs_df)*100:.1f}%)"
        )
        print(f"     Sample sizes: {', '.join(company_size_data.head(3).astype(str))}")
    else:
        print(f"  👥 Company Size Info: Not available (run parser to collect)")

    # Company followers analysis
    company_followers_data = top_jobs_df["company_followers"].dropna()
    if not company_followers_data.empty:
        print(
            f"  📊 Company Followers Info: {len(company_followers_data)}/{len(top_jobs_df)} jobs ({len(company_followers_data)/len(top_jobs_df)*100:.1f}%)"
        )
        print(
            f"     Sample followers: {', '.join(company_followers_data.head(3).astype(str))}"
        )
    else:
        print(f"  📊 Company Followers Info: Not available (run parser to collect)")

    # Company industry analysis
    company_industry_data = top_jobs_df["company_industry"].dropna()
    if not company_industry_data.empty:
        print(
            f"  🏭 Company Industry Info: {len(company_industry_data)}/{len(top_jobs_df)} jobs ({len(company_industry_data)/len(top_jobs_df)*100:.1f}%)"
        )
        industry_counts = company_industry_data.value_counts().head(3)
        print(f"     Top industries: {', '.join(industry_counts.index)}")
    else:
        print(f"  🏭 Company Industry Info: Not available (run parser to collect)")

    # Location distribution (enhanced)
    location_counts = top_jobs_df["location"].value_counts()
    print(f"\n📍 Top Locations:")
    for location, count in location_counts.head().items():
        print(f"  • {location}: {count} job(s)")

    # Work location type analysis
    if "work_location_type" in top_jobs_df.columns:
        work_type_counts = top_jobs_df["work_location_type"].value_counts(dropna=True)
        print(f"\n🏠 Work Location Types (Location Intelligence):")
        for work_type, count in work_type_counts.items():
            emoji = {"Remote": "🏠", "Hybrid": "🔄", "On-site": "🏢"}.get(
                work_type, "📍"
            )
            percentage = count / len(top_jobs_df) * 100
            print(f"  {emoji} {work_type}: {count} job(s) ({percentage:.1f}%)")

    # Experience level distribution
    if "level" in top_jobs_df.columns:
        level_counts = top_jobs_df["level"].value_counts(dropna=True)
        if not level_counts.empty:
            print(f"\n🎯 Experience Levels:")
            for level, count in level_counts.items():
                print(f"  • {level}: {count} job(s)")

    # Employment type distribution
    if "employment_type" in top_jobs_df.columns:
        employment_counts = top_jobs_df["employment_type"].value_counts(dropna=True)
        if not employment_counts.empty:
            print(f"\n💼 Employment Types:")
            for emp_type, count in employment_counts.items():
                print(f"  • {emp_type}: {count} job(s)")

    # Job function analysis
    if "job_function" in top_jobs_df.columns:
        function_counts = top_jobs_df["job_function"].value_counts(dropna=True)
        if not function_counts.empty:
            print(f"\n⚙️ Top Job Functions:")
            for function, count in function_counts.head().items():
                print(f"  • {function}: {count} job(s)")

    # Salary information availability
    salary_jobs = top_jobs_df["salary_range"].notna().sum()
    print(
        f"\n💰 Salary Information: {salary_jobs} out of {len(top_jobs_df)} jobs ({salary_jobs/len(top_jobs_df)*100:.1f}%)"
    )

    # Applicant information
    applicant_jobs = top_jobs_df["applicants"].notna().sum()
    print(
        f"👥 Applicant Count Available: {applicant_jobs} out of {len(top_jobs_df)} jobs ({applicant_jobs/len(top_jobs_df)*100:.1f}%)"
    )

    print(f"\n📈 Data Quality Summary:")
    print(f"  ✅ All jobs have location intelligence classification")
    print(f"  ✅ Enhanced 20-column data structure with company info")
    print(f"  ✅ Company intelligence extraction available")
    print(f"  ✅ Comprehensive job metadata available")

In [ ]:
# Enhanced salary analysis with location and company intelligence
with sqlite3.connect(db_path) as conn:
    salary_query = """
    SELECT title, company, company_size, company_followers, company_industry,
           salary_range, location, work_location_type, level, employment_type
    FROM jobs 
    WHERE salary_range IS NOT NULL AND salary_range != ''
    ORDER BY created_at DESC
    LIMIT 15
    """

    salary_jobs = pd.read_sql_query(salary_query, conn)

if not salary_jobs.empty:
    print("💰 JOBS WITH SALARY INFORMATION + LOCATION & COMPANY INTELLIGENCE")
    print("=" * 75)
    for idx, job in salary_jobs.iterrows():
        # Work type emoji
        work_emoji = {"Remote": "🏠", "Hybrid": "🔄", "On-site": "🏢"}.get(
            job["work_location_type"], "📍"
        )

        print(f"{idx+1:2d}. {job['title']} at {job['company']}")
        print(f"    💰 {job['salary_range']}")
        print(f"    📍 {job['location']} | {work_emoji} {job['work_location_type']}")

        # NEW: Company information display
        company_details = []
        if pd.notna(job["company_size"]) and job["company_size"]:
            company_details.append(f"👥 {job['company_size']} employees")
        if pd.notna(job["company_followers"]) and job["company_followers"]:
            company_details.append(f"📊 {job['company_followers']} followers")
        if pd.notna(job["company_industry"]) and job["company_industry"]:
            company_details.append(f"🏭 {job['company_industry']}")

        if company_details:
            print(f"    🏢 {' | '.join(company_details)}")

        if job["level"]:
            print(f"    🎯 {job['level']}")
        if job["employment_type"]:
            print(f"    📝 {job['employment_type']}")
        print()

    # Salary analysis by work type
    if "work_location_type" in salary_jobs.columns:
        print("📈 SALARY ANALYSIS BY WORK TYPE")
        print("=" * 40)
        work_type_salary = salary_jobs.groupby("work_location_type").size()
        for work_type, count in work_type_salary.items():
            emoji = {"Remote": "🏠", "Hybrid": "🔄", "On-site": "🏢"}.get(
                work_type, "📍"
            )
            print(f"{emoji} {work_type}: {count} jobs with salary info")

    # NEW: Company size analysis for salary jobs
    print(f"\n🏢 COMPANY SIZE ANALYSIS FOR SALARY JOBS")
    print("=" * 45)
    company_size_salary = salary_jobs[salary_jobs["company_size"].notna()]
    if not company_size_salary.empty:
        print(
            f"💼 Jobs with both salary and company size data: {len(company_size_salary)}"
        )
        for idx, job in company_size_salary.head(5).iterrows():
            print(
                f"  • {job['company']}: {job['company_size']} employees | {job['salary_range']}"
            )
    else:
        print("📊 No jobs found with both salary and company size information")
        print(
            "💡 Run 'make run-parser' to collect fresh data with company intelligence"
        )

else:
    print("No jobs with salary information found.")

In [ ]:
# 🎯 LOCATION & COMPANY INTELLIGENCE SHOWCASE
print("🌍 LOCATION & COMPANY INTELLIGENCE ANALYSIS")
print("=" * 60)

with sqlite3.connect(db_path) as conn:
    # Get location intelligence statistics
    location_intel_query = """
    SELECT 
        location,
        work_location_type,
        COUNT(*) as job_count,
        GROUP_CONCAT(DISTINCT company) as companies,
        COUNT(CASE WHEN company_size IS NOT NULL THEN 1 END) as companies_with_size,
        COUNT(CASE WHEN company_industry IS NOT NULL THEN 1 END) as companies_with_industry
    FROM jobs 
    WHERE location IS NOT NULL
    GROUP BY location, work_location_type
    ORDER BY job_count DESC
    LIMIT 10
    """

    location_intel_df = pd.read_sql_query(location_intel_query, conn)

if not location_intel_df.empty:
    print("📊 Location + Work Type + Company Intelligence Distribution:")
    for idx, row in location_intel_df.iterrows():
        emoji = {"Remote": "🏠", "Hybrid": "🔄", "On-site": "🏢"}.get(
            row["work_location_type"], "📍"
        )
        companies = row["companies"].split(",") if row["companies"] else []

        print(
            f"{emoji} {row['location']} - {row['work_location_type']}: {row['job_count']} jobs"
        )
        if len(companies) <= 3:
            print(f"    Companies: {', '.join(companies)}")
        else:
            print(
                f"    Companies: {', '.join(companies[:3])}... (+{len(companies)-3} more)"
            )

        # NEW: Company intelligence stats
        company_intel_info = []
        if row["companies_with_size"] > 0:
            company_intel_info.append(f"👥 {row['companies_with_size']} with size data")
        if row["companies_with_industry"] > 0:
            company_intel_info.append(
                f"🏭 {row['companies_with_industry']} with industry data"
            )

        if company_intel_info:
            print(f"    Company Intel: {' | '.join(company_intel_info)}")
        print()

    # Overall location intelligence summary
    with sqlite3.connect(db_path) as conn:
        summary_query = """
        SELECT 
            work_location_type,
            COUNT(*) as count,
            ROUND(COUNT(*) * 100.0 / (SELECT COUNT(*) FROM jobs), 1) as percentage
        FROM jobs 
        WHERE work_location_type IS NOT NULL
        GROUP BY work_location_type
        ORDER BY count DESC
        """
        summary_df = pd.read_sql_query(summary_query, conn)

    print("🎯 WORK TYPE INTELLIGENCE SUMMARY:")
    print("-" * 40)
    for _, row in summary_df.iterrows():
        emoji = {"Remote": "🏠", "Hybrid": "🔄", "On-site": "🏢"}.get(
            row["work_location_type"], "📍"
        )
        print(
            f"{emoji} {row['work_location_type']:8s}: {row['count']:3d} jobs ({row['percentage']:5.1f}%)"
        )

    # NEW: Company intelligence summary
    with sqlite3.connect(db_path) as conn:
        company_intel_summary = """
        SELECT 
            COUNT(*) as total_jobs,
            COUNT(CASE WHEN company_size IS NOT NULL THEN 1 END) as jobs_with_size,
            COUNT(CASE WHEN company_followers IS NOT NULL THEN 1 END) as jobs_with_followers,
            COUNT(CASE WHEN company_industry IS NOT NULL THEN 1 END) as jobs_with_industry,
            COUNT(CASE WHEN company_size IS NOT NULL AND company_followers IS NOT NULL THEN 1 END) as jobs_with_both
        FROM jobs
        """
        company_stats = pd.read_sql_query(company_intel_summary, conn).iloc[0]

    print(f"\n🏢 COMPANY INTELLIGENCE SUMMARY:")
    print("-" * 40)
    total = company_stats["total_jobs"]
    print(
        f"👥 Company Size Data:     {company_stats['jobs_with_size']:3d}/{total} jobs ({company_stats['jobs_with_size']/total*100:5.1f}%)"
    )
    print(
        f"📊 Company Followers:     {company_stats['jobs_with_followers']:3d}/{total} jobs ({company_stats['jobs_with_followers']/total*100:5.1f}%)"
    )
    print(
        f"🏭 Company Industry:      {company_stats['jobs_with_industry']:3d}/{total} jobs ({company_stats['jobs_with_industry']/total*100:5.1f}%)"
    )
    print(
        f"🎯 Complete Company Data: {company_stats['jobs_with_both']:3d}/{total} jobs ({company_stats['jobs_with_both']/total*100:5.1f}%)"
    )

    print(f"\n✨ Enhanced Intelligence Features:")
    print(f"   🎯 Automatic location extraction from job postings")
    print(f"   🤖 AI-powered work type classification")
    print(f"   🏢 Company size, followers, and industry extraction")
    print(f"   📊 Enhanced analytics with location and company data")
    print(f"   💾 20-column output with integrated company information")

else:
    print(
        "No location data found. Run 'make run-parser' to collect jobs with location & company intelligence."
    )

In [ ]:
# 🔍 QUICK COMPANY INTELLIGENCE CHECK
print("🔍 CURRENT COMPANY INTELLIGENCE COVERAGE")
print("=" * 50)

with sqlite3.connect(db_path) as conn:
    # Get current state of company fields
    coverage_query = """
    SELECT 
        COUNT(*) as total_jobs,
        COUNT(CASE WHEN company_size IS NOT NULL AND company_size != '' THEN 1 END) as jobs_with_size,
        COUNT(CASE WHEN company_followers IS NOT NULL AND company_followers != '' THEN 1 END) as jobs_with_followers,
        COUNT(CASE WHEN company_industry IS NOT NULL AND company_industry != '' THEN 1 END) as jobs_with_industry
    FROM jobs
    """
    coverage_stats = pd.read_sql_query(coverage_query, conn).iloc[0]

    print(f"📊 Database-wide Company Intelligence:")
    total = coverage_stats["total_jobs"]
    print(f"   Total jobs: {total}")
    print(
        f"   👥 Company Size: {coverage_stats['jobs_with_size']} jobs ({coverage_stats['jobs_with_size']/total*100:.1f}%)"
    )
    print(
        f"   📊 Company Followers: {coverage_stats['jobs_with_followers']} jobs ({coverage_stats['jobs_with_followers']/total*100:.1f}%)"
    )
    print(
        f"   🏭 Company Industry: {coverage_stats['jobs_with_industry']} jobs ({coverage_stats['jobs_with_industry']/total*100:.1f}%)"
    )

    # Show some examples of extracted company info
    sample_query = """
    SELECT company, company_size, company_followers, company_industry, title
    FROM jobs 
    WHERE (company_size IS NOT NULL AND company_size != '') 
       OR (company_followers IS NOT NULL AND company_followers != '')
       OR (company_industry IS NOT NULL AND company_industry != '')
    ORDER BY created_at DESC
    LIMIT 10
    """

    sample_companies = pd.read_sql_query(sample_query, conn)

    print(f"\n🏢 Examples of Company Intelligence:")
    for idx, row in sample_companies.iterrows():
        print(f"   {idx+1}. {row['company']}")
        if row["company_size"]:
            print(f"      👥 Size: {row['company_size']}")
        if row["company_followers"]:
            print(f"      📊 Followers: {row['company_followers']}")
        if row["company_industry"]:
            print(f"      🏭 Industry: {row['company_industry']}")
        print(f"      Job: {row['title']}")
        print()

print(f"✨ The enhanced company parser successfully extracted information!")
print(f"💡 To improve coverage further, run: make fix-company-info")

In [ ]:
# 📊 EXPORT & DATA VALIDATION
print("📤 CSV EXPORT WITH ENHANCED DATA + COMPANY INTELLIGENCE")
print("=" * 55)

# Export current job data to CSV in the main data folder
csv_filename = db.export_jobs_to_csv("../data/notebook_analysis_export.csv")
print(f"✅ Jobs exported to: {csv_filename}")

# Validate the exported CSV structure
if csv_filename:
    import pandas as pd

    exported_df = pd.read_csv(csv_filename)

    print(f"\n📋 Export Validation:")
    print(f"   Shape: {exported_df.shape}")
    print(f"   Columns: {exported_df.shape[1]} (should be 21)")

    expected_columns = [
        "id",
        "company",
        "company_size",
        "company_followers",
        "company_industry",
        "title",
        "location",
        "work_location_type",
        "level",
        "salary_range",
        "content",
        "employment_type",
        "job_function",
        "industries",
        "posted_time",
        "applicants",
        "job_id",
        "date",
        "parsing_link",
        "job_posting_link",
        "company_info_link",
    ]

    print(f"\n✅ Column Validation:")
    missing_cols = set(expected_columns) - set(exported_df.columns)
    extra_cols = set(exported_df.columns) - set(expected_columns)

    if not missing_cols and not extra_cols:
        print("   🎯 Perfect! All 21 expected columns present")
    else:
        if missing_cols:
            print(f"   ⚠️  Missing columns: {missing_cols}")
        if extra_cols:
            print(f"   ➕ Extra columns: {extra_cols}")

    print(f"\n📊 Data Quality Check:")
    print(
        f"   Location data: {exported_df['location'].notna().sum()}/{len(exported_df)} jobs ({exported_df['location'].notna().sum()/len(exported_df)*100:.1f}%)"
    )
    print(
        f"   Work type data: {exported_df['work_location_type'].notna().sum()}/{len(exported_df)} jobs ({exported_df['work_location_type'].notna().sum()/len(exported_df)*100:.1f}%)"
    )
    print(
        f"   Company data: {exported_df['company'].notna().sum()}/{len(exported_df)} jobs"
    )
    print(
        f"   Company size: {exported_df['company_size'].notna().sum()}/{len(exported_df)} jobs ({exported_df['company_size'].notna().sum()/len(exported_df)*100:.1f}%)"
    )
    print(
        f"   Company followers: {exported_df['company_followers'].notna().sum()}/{len(exported_df)} jobs ({exported_df['company_followers'].notna().sum()/len(exported_df)*100:.1f}%)"
    )
    print(
        f"   Company industry: {exported_df['company_industry'].notna().sum()}/{len(exported_df)} jobs ({exported_df['company_industry'].notna().sum()/len(exported_df)*100:.1f}%)"
    )

    # Check if company_info_link column exists (since it was recently added)
    if "company_info_link" in exported_df.columns:
        company_link_count = exported_df["company_info_link"].notna().sum()
        print(
            f"   Company info link: {company_link_count}/{len(exported_df)} jobs ({company_link_count/len(exported_df)*100:.1f}%)"
        )

        # DIAGNOSTIC: Show why company_info_link is mostly empty
        if company_link_count == 0:
            print(f"\n🔍 DIAGNOSTIC: Company Info Link Issue")
            print(f"   ❌ No jobs have company_info_link values")
            print(
                f"   🔧 Root Cause: Company URL extraction during parsing not working"
            )
            print(f"   📋 Technical Details:")
            print(
                f"      • Field implementation: ✅ Complete (database schema, models, parser)"
            )
            print(
                f"      • URL extraction: ❌ CSS selectors not finding company links on LinkedIn"
            )
            print(
                f"      • Solution: Update _extract_company_link() method in company_parser.py"
            )
        elif company_link_count < len(exported_df) * 0.1:  # Less than 10%
            print(f"\n⚠️  DIAGNOSTIC: Low Company Info Link Coverage")
            print(f"   📊 Only {company_link_count} jobs have company_info_link")
            print(
                f"   🔧 Likely Issue: CSS selectors partially working but need improvement"
            )
            print(
                f"   📋 Recommendation: Review and update LinkedIn company link selectors"
            )
    else:
        print("   Company info link: ❌ Column missing (database export needs update)")

    print(
        f"   Title data: {exported_df['title'].notna().sum()}/{len(exported_df)} jobs"
    )

    print(
        f"\n🎉 SUCCESS: Enhanced LinkedIn parser with location & company intelligence is working!"
    )
    print(f"   💾 Database: data/jobs.db")
    print(f"   📤 Export: {csv_filename}")
    print(f"   🎯 Use: make run-parser (to collect more jobs with company info)")

print(f"\n" + "=" * 60)
print(
    "🚀 ANALYSIS COMPLETE - Enhanced LinkedIn Parser with Company Intelligence Ready!"
)
print("=" * 60)

In [ ]:
# 🔄 RUN PARSER + CLEANER BACK TO BACK
print("🚀 RUNNING PARSER + DATA CLEANER PIPELINE")
print("=" * 50)

import subprocess
import time

# Step 1: Run the parser to collect fresh job data
print("📥 Step 1: Running LinkedIn Parser...")
print("Command: make run-parser")
try:
    parser_result = subprocess.run(
        ["make", "run-parser"],
        cwd=project_root,
        capture_output=True,
        text=True,
        timeout=300,  # 5 minute timeout
    )

    if parser_result.returncode == 0:
        print("✅ Parser completed successfully!")
        # Extract some stats from output if available
        lines = parser_result.stdout.split("\n")
        for line in lines[-10:]:  # Show last 10 lines
            if line.strip() and (
                "saved" in line.lower()
                or "exported" in line.lower()
                or "jobs" in line.lower()
            ):
                print(f"   {line.strip()}")
    else:
        print(f"⚠️ Parser completed with warnings:")
        print(f"   Return code: {parser_result.returncode}")
        if parser_result.stderr:
            print(f"   Error: {parser_result.stderr[-500:]}")  # Last 500 chars

except subprocess.TimeoutExpired:
    print("⏰ Parser timeout after 5 minutes")
except Exception as e:
    print(f"❌ Parser error: {e}")

# Small delay between operations
time.sleep(2)

# Step 2: Run the data cleaner on the fresh data
print(f"\n🧹 Step 2: Running Data Cleaner...")
print("Command: python -m genai_job_finder.data_cleaner.run_graph")
try:
    cleaner_result = subprocess.run(
        [
            "/home/alireza/.cache/pypoetry/virtualenvs/genai-job-finder-Y_k-9c-5-py3.12/bin/python",
            "-m",
            "genai_job_finder.data_cleaner.run_graph",
            "--db-path",
            "data/jobs.db",
            "--verbose",
        ],
        cwd=project_root,
        capture_output=True,
        text=True,
        timeout=600,  # 10 minute timeout for AI processing
    )

    if cleaner_result.returncode == 0:
        print("✅ Data cleaner completed successfully!")
        # Extract processing summary
        lines = cleaner_result.stdout.split("\n")
        in_summary = False
        for line in lines:
            if "PROCESSING SUMMARY" in line:
                in_summary = True
                print(f"\n📊 {line}")
            elif in_summary and ("=" in line or line.strip() == ""):
                if "=" in line:
                    print(line)
                    in_summary = False
            elif in_summary:
                print(f"   {line}")
    else:
        print(f"⚠️ Data cleaner completed with issues:")
        print(f"   Return code: {cleaner_result.returncode}")
        if cleaner_result.stderr:
            print(f"   Error: {cleaner_result.stderr[-500:]}")

except subprocess.TimeoutExpired:
    print("⏰ Data cleaner timeout after 10 minutes")
except Exception as e:
    print(f"❌ Data cleaner error: {e}")

print(f"\n🎯 Pipeline Complete!")
print("   📥 Fresh job data collected")
print("   🧹 AI-powered data cleaning applied")
print("   💾 Results available in cleaned_jobs table")
print("   📊 Ready for enhanced analysis below ⬇️")

In [ ]:
# 🧹 CLEANED JOBS TABLE ANALYSIS
print("✨ ANALYZING AI-CLEANED JOB DATA WITH COMPANY INTELLIGENCE")
print("=" * 65)

with sqlite3.connect(db_path) as conn:
    # Check if cleaned_jobs table exists
    tables_query = (
        "SELECT name FROM sqlite_master WHERE type='table' AND name='cleaned_jobs'"
    )
    table_exists = pd.read_sql_query(tables_query, conn)

    if table_exists.empty:
        print("❌ No cleaned_jobs table found.")
        print("💡 Run the cell above to execute the parser + cleaner pipeline first.")
    else:
        print("✅ Cleaned jobs table found!")

        # Get basic stats
        total_cleaned = pd.read_sql_query(
            "SELECT COUNT(*) as count FROM cleaned_jobs", conn
        ).iloc[0]["count"]
        print(f"📊 Total cleaned jobs: {total_cleaned}")

        if total_cleaned > 0:
            # Get the schema of cleaned table
            schema_query = "PRAGMA table_info(cleaned_jobs)"
            schema_df = pd.read_sql_query(schema_query, conn)
            print(f"🏗️ Table structure: {len(schema_df)} columns")

            # Sample of cleaned data with company information
            sample_query = """
            SELECT 
                id, company, company_size, company_followers, company_industry,
                title, location, 
                min_years_experience, experience_level_label,
                work_location_type, employment_type,
                min_salary, max_salary, mid_salary, content
            FROM cleaned_jobs 
            ORDER BY id DESC 
            LIMIT 10
            """

            cleaned_sample = pd.read_sql_query(sample_query, conn)

            print(f"\n📋 SAMPLE CLEANED JOBS WITH COMPANY INTELLIGENCE:")
            print("-" * 70)
            for idx, job in cleaned_sample.iterrows():
                print(f"{idx+1:2d}. {job['title']} at {job['company']}")
                print(f"    📍 {job['location']}")

                # NEW: Company information display
                company_details = []
                if pd.notna(job["company_size"]) and job["company_size"]:
                    company_details.append(f"👥 {job['company_size']} employees")
                if pd.notna(job["company_followers"]) and job["company_followers"]:
                    company_details.append(f"📊 {job['company_followers']} followers")
                if pd.notna(job["company_industry"]) and job["company_industry"]:
                    company_details.append(f"🏭 {job['company_industry']}")

                if company_details:
                    print(f"    🏢 {' | '.join(company_details)}")

                # Experience info
                if pd.notna(job["min_years_experience"]) and pd.notna(
                    job["experience_level_label"]
                ):
                    print(
                        f"    🎯 Experience: {job['min_years_experience']} years → {job['experience_level_label']}"
                    )

                # Salary info
                if pd.notna(job["min_salary"]) and pd.notna(job["max_salary"]):
                    print(
                        f"    💰 Salary: ${job['min_salary']:,.0f} - ${job['max_salary']:,.0f} (Mid: ${job['mid_salary']:,.0f})"
                    )

                # Work details
                work_details = []
                if pd.notna(job["work_location_type"]):
                    work_emoji = {"Remote": "🏠", "Hybrid": "🔄", "On-site": "🏢"}.get(
                        job["work_location_type"], "📍"
                    )
                    work_details.append(f"{work_emoji} {job['work_location_type']}")
                if pd.notna(job["employment_type"]):
                    work_details.append(job["employment_type"])
                if work_details:
                    print(f"    📝 {' | '.join(work_details)}")
                print()

cleaned_sample

In [ ]:
# 📊🔄 BEFORE vs AFTER: Data Transformation Analysis with Company Intelligence
print("🔄 ORIGINAL vs AI-CLEANED DATA COMPARISON (WITH COMPANY INTELLIGENCE)")
print("=" * 75)

with sqlite3.connect(db_path) as conn:
    # Check if both tables exist
    original_exists = (
        pd.read_sql_query("SELECT COUNT(*) as count FROM jobs", conn).iloc[0]["count"]
        > 0
    )
    cleaned_exists = (
        len(
            pd.read_sql_query(
                "SELECT name FROM sqlite_master WHERE type='table' AND name='cleaned_jobs'",
                conn,
            )
        )
        > 0
    )

    if not cleaned_exists:
        print("❌ Need cleaned data for comparison")
        print("💡 Run: make run-pipeline")
    elif not original_exists:
        print("❌ No original data found")
    else:
        cleaned_count = pd.read_sql_query(
            "SELECT COUNT(*) as count FROM cleaned_jobs", conn
        ).iloc[0]["count"]

        if cleaned_count == 0:
            print("📭 Cleaned table is empty")
            print("💡 Run: make run-cleaner")
        else:
            print("📊 DATA TRANSFORMATION PIPELINE RESULTS WITH COMPANY INTELLIGENCE:")
            print("-" * 60)

            # Side-by-side comparison of same jobs including company info
            comparison_query = """
            SELECT 
                o.id,
                o.company,
                o.company_size,
                o.company_followers,
                o.company_industry,
                o.title,
                o.location,
                o.level as original_level,
                o.salary_range as original_salary,
                o.employment_type as original_employment,
                c.min_years_experience as ai_years,
                c.experience_level_label as ai_level,
                CASE 
                    WHEN c.min_salary IS NOT NULL THEN c.min_salary || ' - ' || c.max_salary || ' (Mid: ' || c.mid_salary || ')'
                    ELSE 'Not extracted'
                END as ai_salary,
                c.work_location_type as ai_work_type,
                c.employment_type as ai_employment
            FROM jobs o
            LEFT JOIN cleaned_jobs c ON o.id = c.id
            WHERE c.id IS NOT NULL
            ORDER BY o.id DESC
            LIMIT 5
            """

            comparison_df = pd.read_sql_query(comparison_query, conn)

            print("🔍 DETAILED TRANSFORMATION EXAMPLES WITH COMPANY INTELLIGENCE:")
            print("(Showing how AI enhanced the original data)")
            print()

            for idx, row in comparison_df.iterrows():
                print(f"📋 JOB {idx+1}: {row['title']} at {row['company']}")
                print(f"   📍 Location: {row['location']}")

                # NEW: Company intelligence display
                company_details = []
                if pd.notna(row["company_size"]) and row["company_size"]:
                    company_details.append(f"👥 {row['company_size']} employees")
                if pd.notna(row["company_followers"]) and row["company_followers"]:
                    company_details.append(f"📊 {row['company_followers']} followers")
                if pd.notna(row["company_industry"]) and row["company_industry"]:
                    company_details.append(f"🏭 {row['company_industry']}")

                if company_details:
                    print(f"   🏢 Company Intel: {' | '.join(company_details)}")
                print()

                # Experience comparison
                print("   🎯 EXPERIENCE ANALYSIS:")
                print(f"      Original: '{row['original_level'] or 'Not specified'}'")
                print(f"      AI Result: {row['ai_years']} years → {row['ai_level']}")
                print()

                # Salary comparison
                print("   💰 SALARY INTELLIGENCE:")
                print(f"      Original: '{row['original_salary'] or 'Not specified'}'")
                print(f"      AI Result: {row['ai_salary']}")
                print()

                # Employment type comparison
                print("   📝 EMPLOYMENT TYPE:")
                print(
                    f"      Original: '{row['original_employment'] or 'Not specified'}'"
                )
                print(
                    f"      AI Result: {row['ai_employment']} | Work Type: {row['ai_work_type']}"
                )
                print()
                print("-" * 60)

            # Statistical improvements including company intelligence
            print("📈 STATISTICAL IMPROVEMENTS WITH COMPANY INTELLIGENCE:")
            print("-" * 50)

            # Count improvements
            improvements_query = """
            SELECT 
                COUNT(*) as total_jobs,
                -- Experience data
                COUNT(CASE WHEN o.level IS NOT NULL AND o.level != '' THEN 1 END) as original_exp_data,
                COUNT(CASE WHEN c.experience_level_label IS NOT NULL THEN 1 END) as ai_exp_data,
                -- Salary data  
                COUNT(CASE WHEN o.salary_range IS NOT NULL AND o.salary_range != '' THEN 1 END) as original_salary_data,
                COUNT(CASE WHEN c.min_salary IS NOT NULL THEN 1 END) as ai_salary_data,
                -- Work location data
                COUNT(CASE WHEN c.work_location_type IS NOT NULL THEN 1 END) as ai_work_type_data,
                -- Company intelligence data (already in original)
                COUNT(CASE WHEN o.company_size IS NOT NULL THEN 1 END) as company_size_data,
                COUNT(CASE WHEN o.company_followers IS NOT NULL THEN 1 END) as company_followers_data,
                COUNT(CASE WHEN o.company_industry IS NOT NULL THEN 1 END) as company_industry_data
            FROM jobs o
            LEFT JOIN cleaned_jobs c ON o.id = c.id
            WHERE c.id IS NOT NULL
            """

            improvements_stats = pd.read_sql_query(improvements_query, conn).iloc[0]
            total = improvements_stats["total_jobs"]

            print(f"🎯 Experience Data:")
            print(
                f"   Before: {improvements_stats['original_exp_data']}/{total} jobs ({improvements_stats['original_exp_data']/total*100:.1f}%)"
            )
            print(
                f"   After:  {improvements_stats['ai_exp_data']}/{total} jobs ({improvements_stats['ai_exp_data']/total*100:.1f}%)"
            )
            exp_improvement = (
                improvements_stats["ai_exp_data"]
                - improvements_stats["original_exp_data"]
            )
            print(
                f"   Gain:   +{exp_improvement} jobs (+{exp_improvement/total*100:.1f}%)"
            )
            print()

            print(f"💰 Salary Data:")
            print(
                f"   Before: {improvements_stats['original_salary_data']}/{total} jobs ({improvements_stats['original_salary_data']/total*100:.1f}%)"
            )
            print(
                f"   After:  {improvements_stats['ai_salary_data']}/{total} jobs ({improvements_stats['ai_salary_data']/total*100:.1f}%)"
            )
            salary_improvement = (
                improvements_stats["ai_salary_data"]
                - improvements_stats["original_salary_data"]
            )
            print(
                f"   Gain:   +{salary_improvement} jobs (+{salary_improvement/total*100:.1f}%)"
            )
            print()

            print(f"🏠 Work Location Type (New):")
            print(f"   Before: 0/{total} jobs (0.0%) - Not available in original")
            print(
                f"   After:  {improvements_stats['ai_work_type_data']}/{total} jobs ({improvements_stats['ai_work_type_data']/total*100:.1f}%)"
            )
            print(
                f"   Gain:   +{improvements_stats['ai_work_type_data']} jobs (NEW FEATURE)"
            )
            print()

            # NEW: Company intelligence summary
            print(f"🏢 Company Intelligence (Integrated in Parser):")
            print(
                f"   Company Size:     {improvements_stats['company_size_data']}/{total} jobs ({improvements_stats['company_size_data']/total*100:.1f}%)"
            )
            print(
                f"   Company Followers: {improvements_stats['company_followers_data']}/{total} jobs ({improvements_stats['company_followers_data']/total*100:.1f}%)"
            )
            print(
                f"   Company Industry:  {improvements_stats['company_industry_data']}/{total} jobs ({improvements_stats['company_industry_data']/total*100:.1f}%)"
            )
            print(
                "   💡 Company data extracted during parsing phase, available in both tables"
            )

In [ ]:
# 🧪 FRESH DATABASE TEST - Company Info Link Debugging
print("🔬 DIRECT SQL QUERY TEST")
print("=" * 50)

# Test direct SQL query to bypass any caching issues
with sqlite3.connect(db_path) as conn:
    # Test the exact query that should be used in export
    query = '''
        SELECT id, company, title, location, work_location_type, level, salary_range, content,
               employment_type, job_function, industries, posted_time,
               applicants, job_id, date, parsing_link, job_posting_link,
               company_size, company_followers, company_industry, company_info_link
        FROM jobs
        ORDER BY created_at DESC
        LIMIT 5
    '''
    
    try:
        direct_df = pd.read_sql_query(query, conn)
        print(f"✅ Direct SQL query successful:")
        print(f"   Shape: {direct_df.shape}")
        print(f"   Columns: {direct_df.shape[1]}")
        print(f"   company_info_link present: {'company_info_link' in direct_df.columns}")
        
        if 'company_info_link' in direct_df.columns:
            print(f"   Column names: {list(direct_df.columns)}")
            print(f"\n? Sample company_info_link values:")
            for i in range(len(direct_df)):
                company = direct_df.iloc[i]['company']
                link = direct_df.iloc[i]['company_info_link'] 
                status = 'HAS LINK' if pd.notna(link) and link else 'EMPTY'
                print(f"   {company}: {status}")
        else:
            print(f"   ❌ company_info_link column missing from direct query")
            print(f"   Columns: {list(direct_df.columns)}")
            
    except Exception as e:
        print(f"❌ Direct SQL query failed: {e}")

# Now test what the notebook's db instance is actually doing
print(f"\n🔍 NOTEBOOK DB INSTANCE DEBUG:")
try:
    # Check if the database manager has the right database path
    print(f"   Database path: {db.db_path}")
    
    # Test get_all_jobs_as_dataframe method
    test_df = db.get_all_jobs_as_dataframe()
    print(f"   get_all_jobs_as_dataframe(): {test_df.shape}")
    print(f"   Columns: {test_df.shape[1]}")
    print(f"   company_info_link present: {'company_info_link' in test_df.columns}")
    
    # Show what columns are actually returned
    print(f"   Actual columns: {list(test_df.columns)}")
    
except Exception as e:
    print(f"   Error: {e}")

print(f"\n? DIAGNOSIS:")
print(f"   If direct SQL shows 21 columns but db instance shows 20,")
print(f"   then there's an issue with the notebook's database manager instance.")
print(f"   This could be due to an old cached version of the code.")